In [1]:
#@title Импорт библиотек.

# Фиксируем версии библиотек на 22.11.2023
!pip  install  tiktoken==0.5.1
!pip  install  langchain==0.0.339
!pip  install  openai==1.3.4
!pip  install  faiss-cpu==1.7.4
!pip install gspread==3.4.2

from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import MarkdownHeaderTextSplitter

import requests
import os
import re
import getpass
import openai
import tiktoken
from openai import OpenAI
import zipfile
from IPython import display
import timeit

import gspread                  # Импортируем API для работы с Google таблицами
from google.colab import auth   # Импортируем модуль для аутентификации
from google.auth import default # Импортируем модуль для работы с учетными данными

# Очистить экран.
display.clear_output()

In [2]:
#@title Ввод ключа к API OpenAI.

openai.api_key = getpass.getpass("Введите OpenAi API key:")
os.environ["OPENAI_API_KEY"] = openai.api_key

Введите OpenAi API key:··········


# Внутренности. Для любопытных.


In [35]:
#@title Вспомогательные функции

def num_tokens_from_string(string: str) -> int:
    """Возвращает количество токенов в строке"""
    # Выбор кодировщика. `cl100k_base`используется для `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002`
    encoding = tiktoken.get_encoding("cl100k_base")
    # Разбивка строки на токены и подсчет из количества.
    num_tokens = len(encoding.encode(string))
    return num_tokens

def split_text(text, verbose=0):
    """ Функция разбивает текст на чанки. """
    # Шаблон MarkdownHeaderTextSplitter по которому будет делится переданный
    # текст в формате Markdown.
    headers_to_split_on = [ ("#",    "Header 1"),
                            ("##",   "Header 2"),
                            ("###",  "Header 3"),
                            ("####", "Header 4")
                        ]
    # Создаем экземпляр спилиттера.
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    # Получаем список чанков.
    source_chunks = markdown_splitter.split_text(text)

    # Обработка чанков.
    chank_count = len(source_chunks)
    for number, chank in enumerate(source_chunks):
        # Добавление информации в метаданные чанка о его номере в базе.
        chank.metadata["chank"]=f'{number}/{chank_count}'
        # Вывод количества слов/токенов в фрагменте, если включен режим verbose.
        if verbose:
            count = num_tokens_from_string(chank.page_content)
            print(f"\nChank#{number}/{chank_count}. Tokens in text = {count}\n{'-' * 20}\n{insert_newlines(str(chank))}\n{'=' * 20}")

    # Возвращение списка фрагментов текста.
    return source_chunks

def create_embedding(data, verbose=0):
    """Функция преобразует текстовую Базу знаний в векторную."""
    # Разбивка текста на чанки.
    source_chunks = []
    source_chunks = split_text(text=data, verbose=verbose)

    # Создание векторной Базы знаний на основе чанков.
    search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(), )
    # Подсчет общего количества токенов во всех чанках.
    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]))
    # Печать сводной информации по созданию векторной Базы знаний.
    print('\n==================== ')
    print('Количество токенов в документе :', count_token)
    # Стоимость эмбэндинга согласно прайса на 22.11.2023 - 0,0001/1К токенов.
    # https://openai.com/pricing#language-models
    print('ЦЕНА запроса:', 0.0001*(count_token/1000), ' $')
    return search_index

def load_file(url: str):
    """ Функция загрузки документа по url как текст."""
    try:
        response = requests.get(url) # Получение документа по url.
        response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
        return response.text
    except Exception as e:
        print(e)

def load_search_indexes(url: str, verbose=0):
    """Функция загружает текстовую Базу знаний и преобразует ее в векторную."""
    try:
        return create_embedding(load_file(url), verbose=verbose)
    except Exception as e:
        print(e)

def insert_newlines(text: str, max_len: int = 120) -> str:
    """ Функция форматирует переданный текст по длине
    для лучшего восприятия на экране."""
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def filtred_docs (docs, limit_score):
    """ Функция удаляет из отобранных чанков чанки у которых score выше значения limit_score.
    При этом limit_score определяет гарантированно ошибочные чанки.
    Далее отбор чанков идет в зависимости от значения score первого не нулевого
    score. Если есть чанк с score=0 он оставляется единственным.
    Если limit_score = 0, то чанки не фильтруются."""
    if bool(limit_score):
        r = []
        score = 0
        def set_score(d, sc):
            d[0].metadata["score"]=sc
            return d
        for doc in docs:
            s = doc[1]
            if s==0:
                r.append( set_score(doc[0],s))
                break
            if score==0:
                if s<.2:
                    score = s*1.7
                elif s<.3:
                    score = s +.05
                else:
                    score = s +.01
                if score>limit_score:
                        score = limit_score
            if s<score:
                    r.append(set_score(doc,s))
        print(f'Фильр пропустил чанк(ов): {len(r)} из {len(docs)}')
    else:
        r = docs
    return r

def answer_index(model, system, topic: str, query, search_index, temp = 0, verbose_documents = 0,  verbose_price = 0, top_documents = 3, limit_score = 0.0):
    """ Основная функция которая формирует запрос и получает ответ от OpenAI по заданному вопросу
    на основе векторной Базы знаний. """

    # Выбор варианта вопроса. Если есть query, то вопрос задан из группового запроса и он имеет приоритет.
    question = query["question"] if bool(query) else topic
    # Выборка релевантных чанков.
    docs = filtred_docs(search_index.similarity_search_with_score(question, k=top_documents), limit_score)

    """ Этот блок закоментирован. Рассматривается возможность даполнительного
    запроса с фильтрацией чанков по метаданным. Пока нереализовано."""
    # for i, doc in enumerate(docs):
    #     header1 = doc[0].metadata.get('Header 1')
    #     print(f'{i}. {header1}. Score: {doc[1]}')
    #     if header1=='Оценка' or header1=='Экспертиза':
    #         #docs = search_index.similarity_search_with_score(question, k=top_documents, )
    #         #print(f'{i}. {header1}. Score: {doc[1]}')
    #         pass


    message_content = ""            # Контекст для GPT.
    message_content_display = ""    # Контекст для вывода на экран.
    for i, doc in enumerate(docs):
        # Формирование контекста для запроса GPT и показа на экран отобранных чанков.
        message_content = message_content + f'Отрывок документа №{i+1}:{doc[0].page_content}'
        message_content_display = message_content_display + f"\n Отрывок документа №{i+1}. Chank № {doc[0].metadata.get('chank')}. Score({str(round(doc[1], 3))})\n -----------------------\n{insert_newlines(doc[0].page_content)}\n"

        # Сбор информации для группого запроса.
        if bool(query):
            # Выделение из строки метаданных ссылки. Если нет - присваиваем пустую строку.
            search_link_h1 = re.search(r'\[(.*?)\]', doc[0].metadata.get('Header 1')) if bool(doc[0].metadata.get('Header 1')) else ""
            search_link_h2 = re.search(r'\[(.*?)\]', doc[0].metadata.get('Header 2')) if bool(doc[0].metadata.get('Header 2')) else ""
            search_link_h3 = re.search(r'\[(.*?)\]', doc[0].metadata.get('Header 3')) if bool(doc[0].metadata.get('Header 3')) else ""
            search_link_h4 = re.search(r'\[(.*?)\]', doc[0].metadata.get('Header 4')) if bool(doc[0].metadata.get('Header 4')) else ""
            # Выбор самой внутренней ссылки. Если несколько ссылок, то самая внутренняя ссылается на расположение чанка на сайте.
            link = ''
            if bool(search_link_h4):
                link = search_link_h4.group(1)
            elif bool(search_link_h3):
                link = search_link_h3.group(1)
            elif bool(search_link_h2):
                link = search_link_h2.group(1)
            elif bool(search_link_h1):
                link = search_link_h1.group(1)
            # Заполнение запроса выбранными чанками.
            query[f"chank_{i+1}"] = f"Chank № {doc[0].metadata.get('chank')}. Score({str(round(doc[1], 3))}).\n{doc[0].page_content}.\n--------\n{link}"

    # Вывод на экран отобранных чанков.
    if (verbose_documents):
        print(message_content_display)

    # Отправка запроса к Open AI.
    completion = OpenAI().chat.completions.create(
        model = model[0],
        messages = [
            {"role": "system", "content": system } ,
            {"role": "user", "content": f"Документ с информацией для ответа пользователю : {message_content}.\n\nВопрос клиента: {question}"}
        ],
        temperature=temp
    )

    # Подсчет токенов и стоимости.
    prompt_tokens = completion.usage.prompt_tokens
    total_tokens = completion.usage.total_tokens
    price_promt_tokens = prompt_tokens * model[1]/1000
    price_answer_tokens = (total_tokens - prompt_tokens) * model[2]/1000
    price_total_token = price_promt_tokens + price_answer_tokens
    # Сбор информации для группого запроса.
    if bool(query):
        query["price_query"] = price_total_token
        query["price_question"] = price_promt_tokens
        query["price_answer"] = price_answer_tokens
        query["token_query"] = total_tokens
        query["token_question"] = prompt_tokens
        query["token_answer"] = total_tokens - prompt_tokens
    # Вывод на экран стоимости запроса.
    if (verbose_price):
        print('\n======================================================= ')
        print(f'{prompt_tokens} токенов использовано на вопрос. Цена: {round(price_promt_tokens, 6)} $.')
        print(f'{total_tokens - prompt_tokens} токенов использовано на ответ.  Цена: {round(price_answer_tokens, 6)} $.')
        print(f'{total_tokens} токенов использовано всего.     Цена: {round(price_total_token, 6)} $')
        print('======================================================= ')

    # Ответ OpenAI.
    return completion.choices[0].message.content

#question_normalization
def question_normalization(text):
    """ Функция нормализует текст вопроса. Удаляет лишние пробелы,
    символы, знаки. Делает первое слово с заглавной буквы."""

    # Удаление символов "!?.,-" из текста
    #text = re.sub(r'[-!?_]', '', text) # '[-!?.,_]'
    # Разделение текста на слова
    words = text.split()
    # # Проход по каждому слову
    # for i in range(len(words)):
    #     # Если слово состоит из заглавных букв и длина больше 1 символа, считаем это аббревиатурой
    #     if words[i].isupper() and len(words[i]) > 1:
    #         continue  # Пропускаем аббревиатуры
    #     else:
    #         words[i] = words[i].lower()  # Преобразуем в нижний регистр
    # # Преобразуем первое слово в строке к верхнему регистру
    #words[0] = words[0].capitalize()
    # Объединяем слова обратно в строку
    return ' '.join(words)


In [4]:
#@title Вспомогательные функции для загрузки Базы знаний.

def load_bd_text(url: str, verbose=0):
    """ Функция загружает текстовую Базу знаний и
        преобразует ее в векторную."""
    response = requests.get(url) # Получение документа по url.
    response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
    return create_embedding(response.text, verbose=verbose)

def load_bd_vect(url: str, verbose=0):
    """ Функция загружает векторную Базу знаний."""
    name_bd = 'federallab_bd_index.zip'
    # Скачивание архива Базы знаний
    response = requests.get(url) # Получение документа по url.
    response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
    # Сохранение архива.
    with open(name_bd, 'wb') as file:
        file.write(response.content)
    # Разархивирование Базы знаний.
    with zipfile.ZipFile(name_bd, 'r') as zip:
        zip.extractall()
    # Загрузка векторной Базы знаний.
    federallab_bd = FAISS.load_local(f'federallab_bd_index', OpenAIEmbeddings())
    return federallab_bd

def load_bd (url_vect: str, url_text: str, verbose=0):
    """ Функция организует очередность загрузки Базы знаний.
        Сначала идет загрузка векторной базы, если она не загружается,
        то загружается база в текстовом формате и потом преобразуется в векторную."""
    try:
        federallab_bd = load_bd_vect(url_vect, verbose)
        print("Загрузка векторной Базы знаний выполнена успешно.")
        return federallab_bd
    except Exception as e:
        print("По указанной ссылке векторной Базы знаний нет.")
        print(e)
        print("\nИдет загрузка текстовой Базы знаний...")
        try:
            federallab_bd = load_bd_text(url_text, verbose)
            print("\nЗагрузка текстовой Базы знаний выполнена успешно.")
            return federallab_bd
        except Exception as e:
            print("\nПо указанной ссылке текстовой Базы знаний нет.")
            print(e)
            print("\nОшибка загрузки!!")

In [14]:
#@title Архивирование Базы знаний (при необходимости).

# В дальнейшем архив нужно поместить на GitHub
archive = False
if archive:
    folder_to_zip = 'federallab_bd_index'
    output_filename = 'federallab_bd_index_v2.zip'
    # Сохранение папки с векторной Базой знаний.
    federallab_bd_index.save_local(folder_to_zip)
    # Архивирование папки с векторной Базой знаний
    with zipfile.ZipFile(output_filename, 'w') as zip:
        for root, dirs, files in os.walk(folder_to_zip):
            for file in files:
                zip.write(os.path.join(root, file))

# Загрузка Базы знаний.


In [8]:
""" !!! ПАРМЕТРЫ БАЗЫ ЗНАНИЙ !!! """

""" Сначала идет попытка загрузки векторной БЗ, если она неудачна, то идет загрузка текстовой БЗ.

================== ЧИТАЙТЕ СООБЩЕНИЯ ЧТО ЗАГРУЗИЛОСЬ!! ==================================== """

# Сылка на Базу знаний на Github в текстовом формате.
link_bd_text = "https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Datadase/LabSudDB_v1.md"
# Сылка на Базу знаний на Github в векторном формате.
link_bd_vect = 'https://github.com/terrainternship/GPT_labsud/raw/main/federallab_bd_index_v2.zip'

# Показывать полученные чанки только при загрузке Базы знаний в текстовом формате.
verbose_bd = 1

# Загрузка Базы знаний.
federallab_bd_index = load_bd(link_bd_vect, link_bd_text, verbose=verbose_bd )

Загрузка векторной Базы знаний выполнена успешно.


# Параметры запроса нейро-консультанту.

In [10]:
""" !!! ПАРАМЕТРЫ НЕЙРО-КОНСУНТАЛЬТА !!! """

# Данные по названиям модели и стоимости токена на 22.11.2023.
# https://openai.com/pricing#language-models
# Псевдоним = ['Имя модели', 'Цена токена - вопроса', 'Цена токена - ответа'].
MODEL_GPT_4_1106_PREVIEW = ['gpt-4-1106-preview', 0.01, 0.03]   # 128K tokens
MODEL_GPT_3_5_TURBO_1106 = ['gpt-3.5-turbo-1106', 0.001, 0.002] #  16K tokens

# Общие настройки для одиночного и группового запроса.
SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_1106 # Выбранная модель.
top_documents = 3                           # Количество отобранных чанков.
temp = 0                                    # Вариативность ответа.
limit_score = 0                             # Отфильтровывать чанки при поиске выше этого значения, если "0" - не фильтровать.
question_norma = False                      # Нормализовать вопрос. - Удаляет лишние пробелы из вопроса.
#federallab_chat_promt = load_file("https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Dokumov/%D0%A2%D1%8B%20%D1%81%D0%B0%D0%BC%D1%8B%D0%B9%20%D0%BA%D0%BE%D0%BC%D0%BF%D0%B5%D1%82%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9%20%D0%BD%D0%B5%D0%B9%D1%80%D0%BE-%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C.txt")
federallab_chat_promt = load_file("https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Galina/FLSE_promt")

# Настройки только для одиночного запроса.
verbose_documents = 1   # Показывать отобранные чанки.
verbose_price = 1       # Показывать стоимость запроса.

# Одиночный запрос.

In [11]:
#@title Вопрос нейро-консультанту:

""" !!! ВВОД ВОПРОСА !!! """

question = 'Какие задачи позволяет решить финансово-экономическая экспертиза?'

In [12]:
""" <<<< ЖМЕМ. СПРАВОЧНАЯ ИНФОРМАЦИЯ. """

# Проверка нормализации.
print(question)
print(question_normalization(question))

Какие задачи позволяет решить финансово-экономическая экспертиза?
Какие задачи позволяет решить финансово-экономическая экспертиза?


In [22]:
#@title Отправка вопроса нейро-консультанту.

""" <<<< ЖМЕМ """

# Нормализация вопроса при необходимости.
question = question_normalization(question) if question_norma else question
# Запрос GPT
answer = answer_index(
    model = SELECT_MODEL_GPT,
    system = federallab_chat_promt,
    topic = question,
    query = '',
    search_index = federallab_bd_index,
    temp = temp,
    verbose_documents = verbose_documents,
    verbose_price = verbose_price,
    top_documents = top_documents,
    limit_score = limit_score,
)

print()
print(f'ВОПРОС:\n{insert_newlines(question)}\n')
print(f'ОТВЕТ:\n{insert_newlines(answer)}')


 Отрывок документа №1. Chank № 47/219. Score(0.195)
 -----------------------
 Финансово-экономическая экспертиза. Проведение финансово-экономической экспертизы. Финансово-экономической проверка
 направлена на изучение деятельности предприятий и бизнеса с экономической и учетной точки зрения. Проведение
 финансово-экономической экспертизы позволяет решить ряд задач: - выявить и установить факты искажения, полноты и
 правильности отражения информации о хозяйственных операциях в учетных данных; - соответствие фактических данных об
 имуществе и обязательствах организации учетным данным (определение недостачи имущества, расчет фактического размера
 дебиторской и кредиторской задолженности); - анализ финансового состояния организации и отдельных показателей
 деятельности организации, их динамики и причин, обусловивших изменения; - признаки преднамеренности и фиктивности
 банкротства; - справедливости распределения активов и обязательств в процессе реорганизации; - расчет фактически
 получен

# Групповой запрос.
Ячейки выполняем последовательно. Читаем и выбираем внимательно !!

In [24]:
""" <<<< ЖМЕМ """

auth.authenticate_user()        # Аутентифицируем текущего пользователя Colab
creds, _ = default()            # Создаем объект учетных данных на основе аутентификации
gc = gspread.authorize(creds)   # Создаем клиент для таблиц на основе учетных данных

In [25]:
""" <<<< ЖМЕМ """

# Название файла с вопросами.
# Ссылка на общий файл.
try:
    spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1p69Ma_vcEU86_lde61l5RtawJIbGwpAhpXHo2DEKNDo/edit#gid=1834702311')
    print(f'Подключились к документу - {spreadsheet.title}')
except Exception as e:
    print(e)
    print(f'Ошибка подключения к документу. Проверьте ссылку.')

Подключились к документу - FederalLab


In [26]:
""" <<<< ЖМЕМ """

# Получаем список всех страниц файла.
worksheet_list = spreadsheet.worksheets()
print('\n\nСтраницы документа:')
print('-------------------')
for i, worksheet in enumerate(worksheet_list):
        print(f'  {i}. {worksheet.title}')



Страницы документа:
-------------------
  0. Инструкции
  1. Страницы
  2. Исключенные
  3. Докумов
  4. Макеев
  5. Галина
  6. Петрунин
  7. Шляпников
  8. Бугаев
  9. A.Куцинс
  10. Вопросы FineTune


In [27]:
""" !!! ВЫБОР !!! """

# Устанавливаем номер рабочего листа по списку выше.
number_sheet = 6 # Петрунин=6

In [28]:
""" <<<< ЖМЕМ """

# Выбор нужного листа из списка.
worksheet = worksheet_list[number_sheet]
# Проверка текущей страницы.
print(f'  \n\nТекущая страница - "{worksheet.title}"\n')
# Список всех столбцов на странице.
print('  -----№---------Название---')
for i, col in enumerate(worksheet.row_values(1)):
    print(f'  Колонка № {i}. {col}')

  

Текущая страница - "Петрунин"

  -----№---------Название---
  Колонка № 0. Ответственный
  Колонка № 1. URL
  Колонка № 2. Название
  Колонка № 3. Вопрос
  Колонка № 4. Ожидаемый ответ
  Колонка № 5. Ответ GPT
  Колонка № 6. Оценка
  Колонка № 7. Ошибка
  Колонка № 8. Комментарий
  Колонка № 9. Чанк №1
  Колонка № 10. Чанк №2
  Колонка № 11. Чанк №3


In [29]:
""" !!! ВЫБОР !!! """

# Устанавливаем номер колонки с вопросами. Проверить со списком выше.
column_question = 3  # По умолчанию = 3

In [30]:
""" <<<< ЖМЕМ """

# В данном блоке происходит заполнение списка запросов вопросами и дополнительной
# информацией с выбранного листа. Загрузка выполняется ВСЕХ строк с вопросами для
# последующей корректной работы. Далее из этого списка выбирается нужный диапазон
# вопросов для группового запроса к GPT.

# Выбор всех вопросов.
column = worksheet.col_values(column_question)
if question_norma: print("Вопросы будут нормализованы.")
# Создаем пустой список запросов.
list_query = []
# Заполнение списка запросов информацией с выбранного листа.
for i in range(len(column)-1):
    row = worksheet.row_values(i+2)
    # Считывание ячеек с контролем их наличия для избежании ошибки чтения.
    # Проверить индексы row со списком в ячейке выше.
    person = row[0] if len(row)>=1 else ""      # Автор вопроса.
    link = row[1] if len(row)>=2 else ""        # Ссылка на тему.
    subject = row[2] if len(row)>=3 else ""     # Тема вопроса.
    question = row[3] if len(row)>=4 else ""    # Вопрос.
    answer = row[4] if len(row)>=5 else ""      # Ожидаемый ответ.
    appraisal = row[6] if len(row)>=7 else ""   # Оценка.
    # Нормализация вопроса.
    question = question_normalization(question) if question_norma else question
    # Словарь запроса.
    query = {
        "line": i+2,            # Номер строки в документе. Берем строки с вопросами.
        "person": person,       # Автор вопроса.
        "subject": subject,     # Тема вопроса.
        "link": link,           # Ссылка на тему.
        "question": question,   # Вопрос.
        "answer": answer,       # Ожидаемый ответ.
        "answer_gpt": "",       # Ответ GPT.
        "appraisal": appraisal, # Оценка ответа.
        "bug": "",              # Ошибка.
        "comments": "",         # Комментарии.
        "chank_1": "",          # Чанк №1.
        "chank_2": "",          # Чанк №2.
        "chank_3": "",          # Чанк №3.
        "price_query": 0,       # Стоимость запроса общая.
        "price_question": 0,    # Стоимость вопроса с контекстом.
        "price_answer": 0,      # Стоимость ответа.
        "token_query": 0,       # Количество токенов всего вопро-ответ.
        "token_question": 0,    # Количество токенов в вопросе с контекстом.
        "token_answer": 0,      # Количество токенов в ответе.
    }
    list_query.append(query)

print(f'Загрузка списка вопросов завершена. Количество вопросов: {len(column)-1}.')

Загрузка списка вопросов завершена. Количество вопросов: 60.


In [31]:
""" <<<< ЖМЕМ. СПРАВОЧНАЯ ИНФОРМАЦИЯ. """

# Пример считанного первого запроса.
# Установкой среза можно вывести интересующий интервал.
for query in list_query[0:1]:
    print(f'Строка №{query["line"]}. Вопрос: {query["question"]}')
    #print(query)

Строка №2. Вопрос: Какие элементы ТС подвергаются экспертизе техничестого состоянияТС?


In [113]:
""" <<<< ЖМЕМ """

# Диапазон строк с вопросами на выбранной странице.
print(f"Диапазон номеров строк с вопросами (включительно): [{list_query[0]['line']}:{list_query[-1]['line']}].")

Диапазон номеров строк с вопросами (включительно): [2:61].


In [32]:
"""!!! ВЫБОР !!!"""

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first, row_end = 50, 50

In [37]:
#@title Отправка группового запроса нейро-консультанту.

""" <<<< ЖМЕМ """

# Если подвис ответ от GPT можно прервать обработку, установить row_first
# в значение на которой была прервана обработка и запустить обработку заново.
# Ранее обработанные вопросы будут сохранены в файле.

# Обнуление общих затрат на групповой запрос.
total_price_query, total_token_query, total_query = 0, 0, 0
# Фиксация времени.
start_group = timeit.default_timer()

for query in list_query[row_first-2:row_end-1]:
    # Проверка на пустой вопрс. Если пустой пропуск цикла.
    if not bool(query['question'].strip ()):
        print(f'На строке № {query["line"]} - вопроса нет.')
        continue
    # Отправка запроса. Фиксация времени.
    start = timeit.default_timer()
    try:
        query["answer_gpt"]= answer_index(
            model = SELECT_MODEL_GPT,
            system = federallab_chat_promt,
            topic = "",
            query = query,
            search_index = federallab_bd_index,
            temp = temp,
            verbose_documents = 0,
            verbose_price = 0,
            top_documents = top_documents,
            limit_score = limit_score,
        )
        total_price_query += query['price_query']
        total_token_query += query['token_query']
        total_query +=1
    except Exception as e:
        print(f'Ошибка ответа GPT на строке №{query["line"]}. - {e}')
    end = timeit.default_timer()
    # Сообщение об успешности ответа от GPT.
    print(f'Строка №{query["line"]}. Ответ на вопрос получен за - {round(end-start, 3)} сек.')

    # Запись ответа в файл.
    try:
        if bool(query['answer_gpt']):
            worksheet.update_cell(query['line'], 6, query['answer_gpt']) # Ответ GPT.
            worksheet.update_cell(query['line'], 10, query['chank_1'])   # Чанк №1.
            worksheet.update_cell(query['line'], 11, query['chank_2'])   # Чанк №2.
            worksheet.update_cell(query['line'], 12, query['chank_3'])   # Чанк №3.
            print(f'Строка №{query["line"]}. Ответ записан в файл.')
            # Контроль номализации вопроса и перезапись его в таблице.
            if question_norma:
                question_old = worksheet.row_values(query['line'])[column_question]
                if not (question_old == query['question']):
                    print("    Вопрос нормализован и перезаписан в таблице.")
                    print(f"    Старый вариант: {question_old}")
                    print(f"    Новый вариант : {query['question']}")
                    worksheet.update_cell(query['line'], column_question+1, query['question'])
    except Exception as e:
        print('\n========================================================')
        print(f'!!!Ошибка записи строки №{query["line"]} в файл. - {e}')
        print('========================================================')

end_group = timeit.default_timer()
print()
print('-------------------------------------------')
print(f'Вопросов в пакетной обработке - {total_query} шт.')
print(f'Время пакетной обработки      - {round(end_group-start_group, 1)} сек.')
print(f'Стоимость пакетной обработки  - {round(total_price_query, 4)} $.')
print(f'Токенов в пакетной обработке  - {total_token_query} шт.')

Строка №50. Ответ на вопрос получен за - 3.934 сек.
Строка №50. Ответ записан в файл.

-------------------------------------------
Вопросов в пакетной обработке - 1 шт.
Время пакетной обработки      - 5.8 сек.
Стоимость пакетной обработки  - 0.0026 $.
Токенов в пакетной обработке  - 2372 шт.


In [38]:
""" <<<< ЖМЕМ. СПРАВОЧНАЯ ИНФОРМАЦИЯ. """

# Вывод вопросов и ответов.
for i, query in enumerate(list_query):
    if query['answer_gpt']:
        print()
        print(insert_newlines(f"ВОПРОС №{i+2}: {query['question']}"))
        print('---------------------------')
        print(insert_newlines(f"\nОТВЕТ: {query['answer_gpt']}"))
        print('===========================\n')
        print(query['chank_1'])
        print()
        print(query['chank_2'])
        print()
        print(query['chank_3'])


 ВОПРОС №50: Виды налоговой экспертизы
---------------------------
 ОТВЕТ: Лаборатория судебной экспертизы ФЛСЭ проводит налоговую экспертизу, которая включает в себя два направления -
 досудебную и судебную. Досудебная проверка позволяет предупредить судебный процесс и выявить нарушения до его начала, а
 судебная налоговая экспертиза включает анализ финансовой деятельности и документов, поиск доказательств для суда. Кроме
 того, проводится анализ уплаты основных налогов, пеней, штрафов и других санкций, связанных с налоговыми нарушениями,
 на основе бухгалтерской документации, налоговых деклараций, актов проверок и других финансовых документов.

Chank № 33/219. Score(0.2).
Налоговая экспертиза. Этапы проведения налоговой экспертизы.  
Проведение экспертизы включает в себя следующие этапы:  
Оценка налоговых и бухгалтерских документов для оценки налоговых резервов предприятия-заказчика. Проверка выполняется в максимально короткие сроки, результаты формируются в виде экспресс-отчета.
З